<a href="https://colab.research.google.com/github/ozawa-nami/test/blob/main/%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0%E6%B1%BA%E5%AE%9A%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
!pip install selenium
!pip install webdriver_manager pyvirtualdisplay
!apt-get update
!apt-get install -y xvfb wget
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb

from pyvirtualdisplay import Display
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 仮想ディスプレイの起動
display = Display(visible=0, size=(1024, 768))
display.start()

# Chrome のオプションとサービスの設定
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')

s = Service(ChromeDriverManager().install())
#s = Service(ChromeDriverManager("116.0.5845.140").install())


# Chrome ブラウザの起動
driver = webdriver.Chrome(service=s, options=options)






In [ ]:

#正常に動きます。N/A#とか入ってても大丈夫

# 必要なライブラリのインポート
import gspread  # Google Sheetsとの連携に使用します
import google.auth  # 認証情報の取得に使用します
import time  # ウェイト処理に使用します
from google.colab import auth  # Google Colab環境での認証に使用します
from google_auth_oauthlib.flow import InstalledAppFlow  # ローカル環境での認証に使用します
from selenium import webdriver  # ウェブスクレイピングに使用します
from selenium.webdriver.chrome.service import Service  # Chromeドライバの設定に使用します
from webdriver_manager.chrome import ChromeDriverManager  # Chromeドライバの自動インストールに使用します
from bs4 import BeautifulSoup  # HTMLの解析に使用します
from selenium.webdriver.support.ui import WebDriverWait  # 要素の待機に使用します
from selenium.webdriver.support import expected_conditions as EC  # 条件の指定に使用します
from oauth2client.client import GoogleCredentials  # Google APIの認証に使用します
from soupsieve import SelectorSyntaxError


# Googleドライブへの認証を行います
auth.authenticate_user()

# 認証情報を保存するための変数を初期化します
creds = None

# Google Colab環境での認証
if 'google.colab' in str(get_ipython()):
    creds, _ = google.auth.default()
else:
    # ローカル環境の場合、クライアントの秘密ファイルを使用して認証を行います
    # 以下の行の`scraping-384605-1e7cd0ea5328.json`は、Google Cloud Consoleから取得したクライアントの秘密ファイル名に置き換えてください
    flow = InstalledAppFlow.from_client_secrets_file('scraping-384605-1e7cd0ea5328.json', scopes)
    creds = flow.run_local_server(port=0)

# gspreadライブラリに認証情報を設定し、Googleスプレッドシートにアクセスできるようにします
gc = gspread.authorize(creds)

# スクレイピング対象のスプレッドシートのURLとシート名を定義します
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1tGojEuhrflajdU67ewvfyQQCcHybdu08c23CWL4TB94/edit#gid=1454186977"
worksheet_name = "副業エージェント"

# スプレッドシートを開き、シートを指定します
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.worksheet(worksheet_name)

# スプレッドシートからURLとセレクタの情報を読み込みます
url_list = worksheet.range("A6:A13")
selector_list = worksheet.range("B6:B13")

# スクレイピング結果を格納するためのリストを初期化します
results = []

# ChromeのWebドライバを設定し、ブラウザを起動します
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')

s = Service(ChromeDriverManager().install())   #　webdriverとChromeのバージョン合わない時は、直接指定
#ChromeDriverManager("116.0.5845.140").install()
driver = webdriver.Chrome(service=s, options=options)



# 各URLに対してスクレイピングを行います
for i, url_cell in enumerate(url_list):
    url = url_cell.value
    if not url:
        continue  # URLがない場合はスキップする

    # ウェブドライバに指定したURLを開かせます
    try:
        driver.get(url)
    except Exception as e:
        print(f"URL '{url}' の読み込み中にエラーが発生しました: {str(e)}")
        continue

    # ページ上の全ての要素が読み込まれるまで待機します（最大で5秒）
    WebDriverWait(driver, 1).until(EC.presence_of_all_elements_located)

    # ページのHTMLを取得します
    html = driver.page_source.encode('utf-8')

    # BeautifulSoupでHTMLをパースします
    soup = BeautifulSoup(html, 'html.parser')

    # セレクタを取得します
    selector = selector_list[i].value
    if not selector:
        # セレクタが存在しない場合、結果のリストに空文字列を追加します
        results.append(("", url_cell))
        continue



    # セレクタに対応するテキストを取得します
    try:
        text = soup.select_one(selector)
    except SelectorSyntaxError:
        print(f"セレクタ '{selector}' が不適切です。スキップします。")
        continue

    if text:
        try:
            # テキストを浮動小数点数に変換します
            float_val = float(text.text.replace(",", ""))
            results.append((float_val, url_cell))
        except ValueError:
            # テキストが数値に変換できない場合、そのまま結果のリストに追加します
            results.append((text.text, url_cell))
    else:
        # テキストが存在しない場合、結果のリストに空文字列を追加します
        results.append(("", url_cell))


# スクレイピング結果を一括で書き込みます
cells_to_update = []
for result, cell in results:
    cells_to_update.append(gspread.models.Cell(cell.row, 4, result))  # 列に結果を書き込みます（列の列番号を表します）
worksheet.update_cells(cells_to_update)


# ウェブドライバを閉じます
driver.quit()


In [ ]:
#時間かかる場合

# 必要なライブラリのインポート
import gspread  # Google Sheetsとの連携に使用します
import google.auth  # 認証情報の取得に使用します
import time  # ウェイト処理に使用します
from google.colab import auth  # Google Colab環境での認証に使用します
from google_auth_oauthlib.flow import InstalledAppFlow  # ローカル環境での認証に使用します
from selenium import webdriver  # ウェブスクレイピングに使用します
from selenium.webdriver.chrome.service import Service  # Chromeドライバの設定に使用します
from webdriver_manager.chrome import ChromeDriverManager  # Chromeドライバの自動インストールに使用します
from bs4 import BeautifulSoup  # HTMLの解析に使用します
from selenium.webdriver.support.ui import WebDriverWait  # 要素の待機に使用します
from selenium.webdriver.support import expected_conditions as EC  # 条件の指定に使用します
from oauth2client.client import GoogleCredentials  # Google APIの認証に使用します

from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By


# Googleドライブへの認証を行います
auth.authenticate_user()

# 認証情報を保存するための変数を初期化します
creds = None

# Google Colab環境での認証
if 'google.colab' in str(get_ipython()):
    creds, _ = google.auth.default()
else:
    # ローカル環境の場合、クライアントの秘密ファイルを使用して認証を行います
    # 以下の行の`scraping-384605-1e7cd0ea5328.json`は、Google Cloud Consoleから取得したクライアントの秘密ファイル名に置き換えてください
    flow = InstalledAppFlow.from_client_secrets_file('scraping-384605-1e7cd0ea5328.json', scopes)
    creds = flow.run_local_server(port=0)

# gspreadライブラリに認証情報を設定し、Googleスプレッドシートにアクセスできるようにします
gc = gspread.authorize(creds)

# スクレイピング対象のスプレッドシートのURLとシート名を定義します
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1NwFp7_HG64TFHFE7VfppyqDgnMUZ6UI8DYJzR7ETOvc/edit#gid=1064975884"
worksheet_name = "看護師"

# スプレッドシートを開き、シートを指定します
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.worksheet(worksheet_name)

# スプレッドシートからURLとセレクタの情報を読み込みます
url_list = worksheet.range("A5:A81")
selector_list = worksheet.range("B5:B81")

# スクレイピング結果を格納するためのリストを初期化します
results = []

# ChromeのWebドライバを設定し、ブラウザを起動します
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')

s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s, options=options)

# 各URLに対してスクレイピングを行います
for i, url_cell in enumerate(url_list):
    url = url_cell.value
    if not url:
        continue  # URLがない場合はスキップする

    # ウェブドライバに指定したURLを開かせます
    try:
        driver.get(url)
    except Exception as e:
        print(f"URL '{url}' の読み込み中にエラーが発生しました: {str(e)}")
        continue

    # セレクタを取得します
    selector = selector_list[i].value
    if not selector:
        # セレクタが存在しない場合、結果のリストに空文字列を追加します
        results.append(("", url_cell))
        continue

    # ページ上の要素が読み込まれるまで待機します（最大で100秒）
    try:
        WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
    except TimeoutException:
        print(f"URL '{url}' の読み込み中に指定した要素が見つからないため、TimeoutExceptionが発生しました。")
        continue

    # ページのHTMLを取得します
    html = driver.page_source.encode('utf-8')

    # BeautifulSoupでHTMLをパースします
    soup = BeautifulSoup(html, 'html.parser')

    # セレクタに対応するテキストを取得します
    text = soup.select_one(selector)
    if text:
        try:
            # テキストを浮動小数点数に変換します
            float_val = float(text.text.replace(",", ""))
            results.append((float_val, url_cell))
        except ValueError:
            # テキストが数値に変換できない場合、そのまま結果のリストに追加します
            results.append((text.text, url_cell))
    else:
        # テキストが存在しない場合、結果のリストに空文字列を追加します
        results.append(("", url_cell))


# # 各URLに対してスクレイピングを行います
# for i, url_cell in enumerate(url_list):
#     url = url_cell.value
#     if not url:
#         continue  # URLがない場合はスキップする

#     # ウェブドライバに指定したURLを開かせます
#     try:
#         driver.get(url)
#     except Exception as e:
#         print(f"URL '{url}' の読み込み中にエラーが発生しました: {str(e)}")
#         continue

#     # # ページ上の全ての要素が読み込まれるまで待機します（最大で30秒）
#     # WebDriverWait(driver, 100).until(EC.presence_of_all_elements_located)
#     WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))


#     # ページのHTMLを取得します
#     html = driver.page_source.encode('utf-8')

#     # BeautifulSoupでHTMLをパースします
#     soup = BeautifulSoup(html, 'html.parser')

#     # セレクタを取得します
#     selector = selector_list[i].value
#     if not selector:
#         # セレクタが存在しない場合、結果のリストに空文字列を追加します
#         results.append(("", url_cell))
#         continue

#     # セレクタに対応するテキストを取得します
#     text = soup.select_one(selector)
#     if text:
#         try:
#             # テキストを浮動小数点数に変換します
#             float_val = float(text.text.replace(",", ""))
#             results.append((float_val, url_cell))
#         except ValueError:
#             # テキストが数値に変換できない場合、そのまま結果のリストに追加します
#             results.append((text.text, url_cell))
#     else:
#         # テキストが存在しない場合、結果のリストに空文字列を追加します
#         results.append(("", url_cell))


# スクレイピング結果を一括で書き込みます
cells_to_update = []
for result, cell in results:
    cells_to_update.append(gspread.models.Cell(cell.row, 3, result))  # 結果を書き込みます（3はC列の列番号を表します）
worksheet.update_cells(cells_to_update)


# ウェブドライバを閉じます
driver.quit()


URL 'https://www.kango-roo.com/career/search/results.php?work_types%5B%5D=3' の読み込み中に指定した要素が見つからないため、TimeoutExceptionが発生しました。


In [ ]:
#1行ずつ書き出す

import gspread
import google.auth
from google.colab import auth
from google_auth_oauthlib.flow import InstalledAppFlow
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from oauth2client.client import GoogleCredentials
import time

#Googleドライブの認証
auth.authenticate_user()

creds = None
if 'google.colab' in str(get_ipython()):
    creds, _ = google.auth.default()
else:
    flow = InstalledAppFlow.from_client_secrets_file('scraping-384605-1e7cd0ea5328.json', scopes)
    creds = flow.run_local_server(port=0)

gc = gspread.authorize(creds)

#スプレッドシートのURLとシート名
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1D9En1wvE4BfIM3ptWD-AdcB4n4JeSPEGn287xDViSNs/edit#gid=295107686"
worksheet_name = "スクレイピング"

#スプレッドシートの読み込み
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.worksheet(worksheet_name)

#URLとセレクタの取得　A5以下にURL指定、B5セル以下にセレクタ指定
url_list = worksheet.range("A5:A288")
selector_list = worksheet.range("B5:B288")

#空のリストの作成
results = []

#ブラウザの起動
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')

s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s, options=options)

# 各URLに対してスクレイピング
for i, url_cell in enumerate(url_list):
    # URLの取得
    url = url_cell.value
    if not url:
        break

    # URLの確認
    print(url)

    # リクエストの送信
    driver.get(url)

    # ページ上のすべての要素が読み込まれるまで待機（15秒でタイムアウト判定）
    WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located)

    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

    # # セレクタの取得
    # selector = selector_list[i].value
    # if not selector:
    #     worksheet.update_cell(i+5, 3, "")  # C列に空文字列を書き込む
    #     continue

  # セレクタの取得
    selector = selector_list[i].value
    if not selector or selector == '#N/A':
        worksheet.update_cell(i+5, 3, "")  # C列に空文字列を書き込む
        continue



    # テキストの取得
    text = soup.select_one(selector)
    if text:
        try:
            float_val = float(text.text.replace(",", ""))  # 数値として扱える場合は、float型に変換
            worksheet.update_cell(i+5, 23, float_val)  # C列に数値を書き込む
        except ValueError:
            worksheet.update_cell(i+5, 23, text.text)  # 数値でない場合は、文字列として書き込む
        time.sleep(1)  # Google Sheets APIの書き込みリミットを超えないように1秒待つ
    else:
        worksheet.update_cell(i+5, 23, "")  # C列に空文字列を書き込む
        time.sleep(1)  # Google Sheets APIの書き込みリミットを超えないように1秒待つ


    # 指定された待ち時間だけ処理を停止
    sleeptime = 2  # 待ち時間を指定（単位は秒）
    time.sleep(sleeptime)

# ブラウザを終了
driver.quit()

https://doda.jp/DodaFront/View/JobSearchList.action?sid=TopSearch&usrclk=PC_logout_kyujinSearchArea_searchButton
https://www.jaic-g.com/
https://www.r-agent.com/kensaku/kinmuchi/?rndmprm=04541440461742190157587242902320230331155405764
https://www.r-agent.com/kensaku/gyoukai/industry1-01/
https://directscout.recruit.co.jp/offers/
https://mynavi-job20s.jp/job/o/list
https://doda-x.jp/job/search/
https://career.levtech.jp/engineer/offer/search/?top_type=100&job_specify=none&keyword=
https://www.kango-roo.com/career/search/results.php?emp_types%5B%5D=1
https://kango-oshigoto.jp/search/%E5%B8%B8%E5%8B%A4(%E5%A4%9C%E5%8B%A4%E6%9C%89%E3%82%8A)-%E6%97%A5%E5%8B%A4%E5%B8%B8%E5%8B%A4/
https://www.nursejinzaibank.com/
https://next.rikunabi.com/search/
https://mc-nurse.net/jobs/search/
https://iishuusyoku.com/03/03-03-1.php
https://www.ptotjinzaibank.com/pt/search_result/1?job_type=5&free_keyword=
https://iryouworker.com/search/search/?workstyle_id%5B%5D=1&workstyle_id%5B%5D=2&workstyle_id%5B%5D=3&

WebDriverException: ignored

In [ ]:
#見出しの取得

!pip install gspread beautifulsoup4 requests

import gspread
import google.auth
from google.colab import auth
from bs4 import BeautifulSoup
import requests
import time
from oauth2client.client import GoogleCredentials

# Googleドライブの認証
auth.authenticate_user()

creds, _ = google.auth.default()

gc = gspread.authorize(creds)

# スプレッドシートのURLとシート名
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1boKdEkPqZS_LMISwwYb0UtamwR1JrxD7FQjH7kQ3hoQ/edit#gid=0"
worksheet_name = "シート1"

# スプレッドシートの読み込み
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.worksheet(worksheet_name)

# URLの取得　A5以下にURL指定
url_cells = worksheet.col_values(1)[4:]  # Pythonのリストは0から始まるため、5行目はインデックス4に相当します。

# 各URLに対してスクレイピング
for i, url in enumerate(url_cells, start=5):  # 開始番号を5に設定
    if not url:
        worksheet.update_cell(i, 3, "")  # C列に空文字を書き込む
        continue

    # リクエストの送信
    try:
        res = requests.get(url)
        res.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request error for {url}: {e}")
        worksheet.update_cell(i, 3, "")  # C列に空文字を書き込む
        continue

    # ページの解析
    soup = BeautifulSoup(res.text, 'html.parser')
    # H2とH3の見出しの取得
    headers = soup.find_all(['h2', 'h3'])

    # H2, H3見出しを整頓し、リストに追加
    headers_list = []
    for h in headers:
      if h.name == 'h2':
          headers_list.append(f"H2: {h.text}")
      elif h.name == 'h3':
        headers_list.append(f"H3: {h.text}")

    # ソース上の順番を崩さず、書き込み規則に従った文字列を作成
    headers_text = "\n".join(headers_list)


    # 結果をスプレッドシートに書き込む
    worksheet.update_cell(i, 3, headers_text)

    # APIの利用制限を考慮して待ち時間を設ける
    time.sleep(2)

In [ ]:
# 必要なライブラリのインストール
#!pip install gspread beautifulsoup4 requests

import gspread
import google.auth
from google.colab import auth
from bs4 import BeautifulSoup
import requests
import time
import re
from oauth2client.client import GoogleCredentials

# Googleドライブの認証
auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# スプレッドシートのURLとシート名
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1boKdEkPqZS_LMISwwYb0UtamwR1JrxD7FQjH7kQ3hoQ/edit#gid=0"
worksheet_name = "シート1"

# スプレッドシートの読み込み
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.worksheet(worksheet_name)

# URLの取得 A5セル以下にURL指定
url_cells = worksheet.col_values(1)[4:]

# 各URLに対してスクレイピングと情報取得
for i, url in enumerate(url_cells, start=5):
    if not url:
        worksheet.update_cell(i, 2, "")  # B列に空文字を書き込む
        worksheet.update_cell(i, 3, "")  # C列に空文字を書き込む
        worksheet.update_cell(i, 4, "")  # D列に空文字を書き込む
        worksheet.update_cell(i, 5, "")  # E列に空文字を書き込む
        worksheet.update_cell(i, 6, "")  # F列に空文字を書き込む
        worksheet.update_cell(i, 7, "")  # G列に空文字を書き込む
        continue

    # リクエストの送信
    try:
        res = requests.get(url)
        res.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request error for {url}: {e}")
        worksheet.update_cell(i, 2, "")  # B列に空文字を書き込む
        worksheet.update_cell(i, 3, "")  # C列に空文字を書き込む
        worksheet.update_cell(i, 4, "")  # D列に空文字を書き込む
        worksheet.update_cell(i, 5, "")  # E列に空文字を書き込む
        worksheet.update_cell(i, 6, "")  # F列に空文字を書き込む
        worksheet.update_cell(i, 7, "")  # G列に空文字を書き込む
        continue

    # ページの解析
    soup = BeautifulSoup(res.text, 'html.parser')


    # H1タイトル
    h1_tag = soup.find('h1')
    h1_title = h1_tag.text.strip() if h1_tag else "N/A"

    # H2とH3の見出しの取得
    headers = soup.find

    # H2とH3の見出しの取得
    headers = soup.find_all(['h2', 'h3'])

    # H2, H3見出しを整頓し、リストに追加
    headers_list = []
    for h in headers:
        if h.name == 'h2':
            headers_list.append(f"H2: {h.text.strip()}")
        elif h.name == 'h3':
            headers_list.append(f"H3: {h.text.strip()}")

    # ソース上の順番を崩さず、書き込み規則に従った文字列を作成
    headers_text = "\n".join(headers_list)

    # サイトの文字数
    site_text = soup.get_text()
    site_text_length = len(re.sub(r'\s+', ' ', site_text))

    # ディスクリプション（metaタグから取得）
    description = soup.find('meta', attrs={'name': 'description'})
    description_content = description.get('content').strip() if description else "N/A"

    # 総リンク数のカウント
    all_links = soup.find_all('a')
    total_link_count = len(all_links)

    # 画像数のカウント
    images = soup.find_all('img')
    image_count = len(images)

    # 出力
    worksheet.update_cell(i, 2, h1_title)  # B列にタイトルを書き込む
    worksheet.update_cell(i, 3, headers_text)  # C列にH2H3情報を書き込む
    worksheet.update_cell(i, 4, site_text_length)  # D列にサイトの文字数を書き込む
    worksheet.update_cell(i, 5, description_content)  # E列にサイトのディスクリぷしょんを書き込む
    worksheet.update_cell(i, 6, total_link_count)  # F列に総リンク数を書き込む
    worksheet.update_cell(i, 7, image_count)  # G列に画像数を書き込む

    # APIの利用制限を考慮して待ち時間を設ける
    time.sleep(2)
